In [2]:
import cv2
import pickle
import mediapipe as mp

In [3]:
import numpy as np

In [4]:
#Initialising mediapipe for getting the hand landmarks
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3,max_num_hands=2)

In [5]:
model_dict = pickle.load(open('./model.pkl', 'rb'))
model = model_dict['model']

Testing Opencv

In [8]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()

    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Creaing a cap obect that uses the camera to detect the hand sign

In [10]:
cap = cv2.VideoCapture(0)
labels_dict = {0: 'A', 1: 'B', 2: 'C',3: 'D',4: 'E', 5: 'F',  6: 'G', 7:'H', 8:'I',9:'J'} #dictiionary containg label names
while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame, #image to draw
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
        
        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                data_aux.append(x)
                data_aux.append(y)
                x_.append(x)
                y_.append(y)

        x1 = int(min(x_)*W) - 15
        y1 = int(min(y_)*H) - 15

        x2 = int(max(x_)*W) - 15
        y2 = int(max(y_)*H) - 15

        prediction = model.predict([np.asarray(data_aux)])

        predicted_char = labels_dict[int(prediction[0])]

        # print(predicted_char)
        # cv2.rectangle(frame,(x1,y1), (x2,y2), (0,0,0), 4)
        cv2.putText(frame,predicted_char,(x1,y1), cv2.FONT_HERSHEY_SIMPLEX,1.3,(0,0,0), 3,cv2.LINE_AA) #printing the detected alphabet

    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()